In [7]:
import tensorflow as tf
import numpy as np

from nems.layers import WeightChannels

wc = WeightChannels(shape=(18,4))
wc_tf = wc.as_tensorflow_layer()()

In [20]:
wc_tf.weights[0].name

'coefficients:0'

In [17]:
x = np.random.rand(10, 1000, 18)  # 10 stimuli (as spectrogram) 1000 bins each
y = np.random.rand(10, 1000, 1)   # 10 corresponding responses
data = tf.data.Dataset.from_tensor_slices((x,y))

<TensorSliceDataset shapes: ((1000, 18), (1000, 1)), types: (tf.float64, tf.float64)>